# 1 Quick Cats v Dogs

In [1]:
from fastai.conv_learner import *
PATH = "data/dogscats/"
sz=224; bs=64

In [2]:
# 1 - how do we want to transform our data
tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
# 2 - get the data
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs) 
#note there is a test_name which will need to be populated for Kaggle submissions
# 3- create the model using pretrained arcitecture
learn = ConvLearner.pretrained(resnet50, data)
# 4 - fit with all layers frozen expect the last layer
%time learn.fit(1e-2, 3, cycle_len=1) # 3 cycles of length 1

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/paperspace/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:24<00:00, 4188208.44it/s]


epoch      trn_loss   val_loss   accuracy                     
    0      0.037291   0.027565   0.989     
    1      0.047995   0.026472   0.99                         
    2      0.036208   0.026972   0.992                        

CPU times: user 15min 8s, sys: 4min 22s, total: 19min 30s
Wall time: 6min 5s


[array([0.02697]), 0.992]

In [3]:
learn.unfreeze() # train the whole thing
learn.bn_freeze(True)
%time learn.fit([1e-5, 1e-4, 1e-2], 1, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      0.021527   0.023947   0.991     

CPU times: user 6min 40s, sys: 2min 24s, total: 9min 5s
Wall time: 4min 40s


[array([0.02395]), 0.991]

In [14]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs) #its a numpu array sp use accuracy_np

(0.9925, 0.017198676601252138)